In [ ]:
#DATA 
# to import data from google map - export map as KML then convert to geojson using an online converter
# each category of points will be a file
# census data from downloaded csvs off of the data table explorers
# converted to a more logical format by hand

# %pip install --force-reinstall -v "folium==0.17.0"
# %pip install geopandas
# %pip install pygris

# %pip install git+https://github.com/JGreenlee/e-mission-common.git@0.6.2

# %pip install urllib3==1.26.6
# %pip install boto3

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import folium
import pygris as pygris
import folium.plugins as fpl
import numpy as np
import branca.colormap as cm

import emcommon.diary.base_modes as emcb
import emcommon.metrics.footprint.util as emcfu
import emcommon.metrics.footprint.footprint_calculations as emcfc

In [ ]:
import logging
logging.getLogger("fiona.collection").setLevel(logging.WARNING)
logging.getLogger("fiona").setLevel(logging.WARNING)

In [ ]:
# add parks and reserves 
parks = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Parks_and_Reserves.geojson")
# add libraries
libraries = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Libraries.geojson")
#add medical centers
medical_centers = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Hospitals_and_Clinics.geojson")
# add high schools
city_HS = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Vernon_Cty_High_Schools.geojson")
# add villages and cities w/o charges
no_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Villages_and_Cities_without_charging.geojson")
# mark L2 chargers
l2_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Existing_L2_Chargers.geojson")
# mark DC fast chargers
dc_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Existing_DC_Fast_Chargers.geojson")
# mark potential sites
potential_dc_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Potential_DC_Fast_Chargers.geojson")

#stitch the dfs together
parks['color'] = "darkgreen"
parks['icon'] = "tent"
libraries['color'] = "blue"
libraries['icon'] = "book"
medical_centers['icon'] = "hospital"
medical_centers['color'] = "darkred"
city_HS["color"] = "lightgreen"
city_HS["icon"] = "school"
l2_charger["color"] = "orange"
l2_charger["icon"] = "plug"
dc_charger["color"] = "lightred"
dc_charger["icon"] = "bolt"
no_charger["color"] = "gray"
no_charger["icon"] = "x"
potential_dc_charger["color"] = "lightblue"
potential_dc_charger["icon"] = "question"

# provided_POIs = pd.concat([parks, libraries, medical_centers, city_HS, l2_charger, dc_charger, no_charger, potential_dc_charger], axis=0)
POIs = [parks, libraries, medical_centers, city_HS, l2_charger, dc_charger, no_charger, potential_dc_charger]
names = ["Parks", "Libraries", "Medical Centers", "High Schools", "L2 Chargers", "DC Chargers", "No Charger", "Potential DC Charger"]

In [ ]:
wi_counties = pygris.counties(state="WI", cache=True)
vernon_outline = wi_counties[wi_counties['NAME'] == "Vernon"]
vernon_outline = vernon_outline.to_crs(epsg=4326)
vernon_outline

In [ ]:
from pygris.data import get_lodes
wi_od_tract = get_lodes(state = "WI", year = 2021, lodes_type = "od", cache = True)

vernon_lodes = wi_od_tract[wi_od_tract['w_geocode'].str.slice(stop = 5) == "55123"]
vernon_lodes["w_block_group"] = vernon_lodes["w_geocode"].str.slice(stop=12)
vernon_lodes = vernon_lodes.groupby("w_block_group").sum()[["S000"]]
vernon_jobs = vernon_lodes[["S000"]]
wi_blocks = pygris.blocks(state="WI", county="Vernon", cache=True)
from pygris.data import get_lodes

#get the lodes data
wi_od_tract = get_lodes(state = "WI", year = 2021, lodes_type = "od", cache = True)

# #filter to Vernon County
vernon_lodes = wi_od_tract[wi_od_tract['h_geocode'].str.slice(stop = 5) == "55123"]
wi_blocks = gpd.GeoDataFrame(wi_blocks, geometry="geometry")

#joint the logdes data to geometry
lodes_geo = vernon_lodes.merge(wi_blocks, how="left", left_on="w_geocode", right_on="GEOID20")
lodes_geo = lodes_geo.merge(wi_blocks, how="left", left_on="h_geocode", right_on="GEOID20")

#get center of each block (to calc commute)
lodes_geo = lodes_geo.dropna(subset=["geometry_x"])
lodes_geo["w_center"] = lodes_geo["geometry_x"].apply(lambda p: p.centroid)
lodes_geo = lodes_geo.dropna(subset=["geometry_y"])
lodes_geo["h_center"] = lodes_geo["geometry_y"].apply(lambda p: p.centroid)
#project coordinates
lodes_geo = gpd.GeoDataFrame(lodes_geo, geometry="w_center", crs="EPSG:4326") #basic coordinates
lodes_geo = lodes_geo.to_crs('EPSG:3071') #meters, good for Wisconsin
lodes_geo = gpd.GeoDataFrame(lodes_geo, geometry="h_center", crs="EPSG:4326")
lodes_geo = lodes_geo.to_crs('EPSG:3071')
#calculate commute, convert to miles
lodes_geo["approx_commute"] = lodes_geo.apply(lambda r: r["w_center"].distance(r["h_center"]), axis = 1)
lodes_geo["commute_miles"] = lodes_geo.apply(lambda r: r["approx_commute"] / 1609.34, axis = 1)

#commute "home" distances
home_commute = lodes_geo[["h_geocode","commute_miles", "S000"]]
home_commute["h_block_group"] = home_commute["h_geocode"].str.slice(stop=12)
home_commute = home_commute[["h_block_group", "commute_miles", "S000"]]
home_commute = home_commute.groupby("h_block_group").sum()
home_commute["commute_miles"] = home_commute["commute_miles"] / home_commute["S000"]

#commute "work" distance
work_commute = lodes_geo[["w_geocode","commute_miles", "S000"]]
work_commute["w_block_group"] = work_commute["w_geocode"].str.slice(stop=12)
work_commute = work_commute[["w_block_group", "commute_miles", "S000"]]
work_commute = work_commute.groupby("w_block_group").sum()
work_commute["commute_miles"] = work_commute["commute_miles"] / work_commute["S000"]

wi_block_groups = pygris.block_groups(state="WI", county="Vernon", cache=True)
wi_block_groups.head()

test_df = wi_block_groups.merge(work_commute, how="left", left_on="GEOID", right_on="w_block_group")
test_df = test_df.rename(columns={"commute_miles": "workplace_commute_length"})
test_df = test_df.merge(home_commute, how="left", left_on = "GEOID", right_on="h_block_group")
test_df = test_df.rename(columns={"commute_miles": "home_commute_length"})
test_df = test_df.merge(vernon_jobs, how="left", left_on="GEOID", right_on="w_block_group")

test_df.dropna(subset = ["S000", "home_commute_length", "workplace_commute_length"], how = "all", inplace=True)

In [ ]:
async def get_commute_data(row, mode, distance_col):
  mode_footprint = emcb.get_rich_mode(mode)["footprint"]
  distance = row[distance_col] * 1609.34
  year = 2024
  coords = row["geometry"].centroid
  coords = [coords.x, coords.y]
  uace = None
  egrid_region = await emcfu.get_egrid_region(coords, year)
  passengers = 1
  
  footprint = await emcfc.calc_footprint(mode_footprint, distance, year, coords, uace, egrid_region, passengers)

  return footprint

for index, row in test_df.iterrows():
  car = await get_commute_data(test_df.iloc[index], {"value":"drove_alone", "base_mode":"CAR", "passengers": 1 }, "workplace_commute_length")
  test_df.loc[index, "work_car"] = car[:1]
  ecar = await get_commute_data(test_df.iloc[index], {"value":"e_car_drove_alone", "base_mode":"E_CAR", "passengers": 1 }, "workplace_commute_length")
  test_df.loc[index, "work_ecar"] = ecar[:1]

  car = await get_commute_data(test_df.iloc[index], {"value":"drove_alone", "base_mode":"CAR", "passengers": 1 }, "home_commute_length")
  test_df.loc[index, "home_car"] = car[:1]
  ecar = await get_commute_data(test_df.iloc[index], {"value":"e_car_drove_alone", "base_mode":"E_CAR", "passengers": 1 }, "home_commute_length")
  test_df.loc[index, "home_ecar"] = ecar[:1]

#calculate energy and emissions saved "if an average worker in the block group commuted in an ev instead of a gas car"
test_df["workplace_e_saved"] = test_df.apply(lambda x : x["work_car"][0]["kwh"] - x["work_ecar"][0]["kwh"], axis = 1)
test_df["workplace_co2_saved"] = test_df.apply(lambda x : x["work_car"][0]["kg_co2"] - x["work_ecar"][0]["kg_co2"], axis = 1)

#calculate energy and emissions saved "if an average worker in the block group commuted in an ev instead of a gas car"
test_df["home_e_saved"] = test_df.apply(lambda x : x["home_car"][0]["kwh"] - x["home_ecar"][0]["kwh"], axis = 1)
test_df["home_co2_saved"] = test_df.apply(lambda x : x["home_car"][0]["kg_co2"] - x["home_ecar"][0]["kg_co2"], axis = 1)

#round values for display
test_df["workplace_e_saved"] = round(test_df["workplace_e_saved"], 2)
test_df["workplace_co2_saved"] = round(test_df["workplace_co2_saved"], 2)
test_df["home_e_saved"] = round(test_df["home_e_saved"], 2)
test_df["home_co2_saved"] = round(test_df["home_co2_saved"], 2)
test_df["home_commute_length"] = round(test_df["home_commute_length"], 2)
test_df["workplace_commute_length"] = round(test_df["workplace_commute_length"], 2)


In [ ]:
#add census tract data
census_income = pd.read_csv("census_income.csv")
census_pop = pd.read_csv("census_population.csv")
census_hous = pd.read_csv("census_housing.csv")

tracts = pygris.tracts(state="WI", cache=True)
tracts = tracts.to_crs(epsg=4326)
tracts = tracts[tracts["COUNTYFP"] == "123"]
tracts["NAME"] = tracts["NAME"].apply(int)

tract_income = tracts.merge(census_income, how="left", left_on="NAME", right_on="Census Tract")
tract_in_pop = tract_income.merge(census_pop, how="left", on="Census Tract")
tract_in_pop_hous = tract_in_pop.merge(census_hous, how="left", on="Census Tract")
tract_in_pop_hous["Percent Multifamily"] = round(((tract_in_pop_hous["Multi-Unit"])/(tract_in_pop_hous["Occupied housing units"])) * 100, 2)


In [ ]:
#https://geopandas.org/en/stable/gallery/plotting_with_folium.html
map_vernon = folium.Map([43.44476, -90.76852], zoom_start = 10,tiles='https://tile.openstreetmap.org/{z}/{x}/{y}.png', attr='My Data Attribution')

#add markers from Google Map
x = 0
for poi in POIs:
    # Create a geometry list from the GeoDataFrame
    geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in poi.geometry]
    fg = folium.FeatureGroup(name=names[x], show="True").add_to(map_vernon)

    x = x + 1
    
    i=0
    for coordinates in geo_df_list:
        # icon = fpl.BeautifyIcon(
        #     icon=provided_POIs.iloc[i]['icon'], border_color=provided_POIs.iloc[i]['color'], text_color=provided_POIs.iloc[i]['color'], icon_shape="square", inner_icon_style='margin-top:0;'
        # )
        # Place the markers with the popup labels and data
        description = str(poi.iloc[i]['description']) if "description" in poi.iloc[i].keys() else ""
        folium.Marker(
                location=coordinates,
                popup=folium.Popup("Name: "
                + str(poi.iloc[i]['Name'])
                + "<br>"
                + "Description: "
                + str(description)),
                # icon=icon
                icon = folium.Icon(color="%s" % poi.iloc[i]['color'])
            ).add_to(fg)
        i = i + 1

#add the county line
fg = folium.FeatureGroup(name="County Line", show="True").add_to(map_vernon)
#add the county outline
#https://geopandas.org/en/stable/gallery/polygon_plotting_with_folium.html
for _, r in vernon_outline.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "transparent"})
    geo_j.add_to(fg)

values = tract_in_pop["Median Household income (dollars)"].unique()
cmap = cm.linear.RdPu_05.scale(min(values), max(values))

#add the census demographics
fg = folium.FeatureGroup(name="Census Demographics", show="True").add_to(map_vernon)
for _, r in tract_in_pop_hous.iterrows():
  inc = r["Median Household income (dollars)"]
  sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
  geo_j = sim_geo.to_json()
  geo_j = folium.GeoJson(data=geo_j, style_function=lambda x, inc=inc: {
    'fillColor': cmap(inc)
  })
  html = ("Census Tract ID: " + str(r["GEOID"]) +
               "<br>Inc: " + str(r["Median Household income (dollars)"]) + 
               "<br>Pop: " + str(r["Population"]) + 
               "<br>% MultiFamily: " + str(r["Percent Multifamily"]) + 
               "<br>Owned Units: " + str(r["Owner-occupied"]) + 
               "<br>Owned HH Size: " + str(r["Average household size of owner-occupied unit"]) +
               "<br>Rented Units: " + str(r["Renter-occupied"]) +
               "<br>Rented HH Size: " + str(r["Average household size of renter-occupied unit"]))
  iframe = folium.IFrame(html)
  folium.Popup(iframe, min_width = 250, max_width = 500).add_to(geo_j)
  geo_j.add_to(fg)

values2 = test_df["workplace_e_saved"].unique()
cmap2 = cm.linear.RdPu_05.scale(min(values2), max(values2))

#add the lodes data
fg = folium.FeatureGroup(name="LODES Jobs", show="True").add_to(map_vernon)
for _, r in test_df.iterrows():
  e = r["workplace_e_saved"]
  sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
  geo_j = sim_geo.to_json()
  # geo_j = folium.GeoJson(data=geo_j)
  geo_j = folium.GeoJson(data=geo_j, style_function=lambda x, e=e: {
    'fillColor': cmap2(e)
  })
  html = ("Block Group ID: " + str(r["GEOID"]) +
          "<br>Jobs: " + str(r["S000"]) + 
          "<br>Residents -> Work: " + str(r["home_commute_length"]) + "miles"
          "<br>Worker -> Home: " + str(r["workplace_commute_length"]) + "miles"
          "<br>Avg Resident saves: " + str(r["home_e_saved"]) + " kwh and " + str(r["home_co2_saved"]) + "kg co2 daily"+
          "<br>Avg Worker saves: " + str(r["workplace_e_saved"]) + " kwh and " + str(r["workplace_co2_saved"]) + "kg co2 daily"
          )
  # html = ("hello world")
  iframe = folium.IFrame(html)
  folium.Popup(iframe, min_width = 350, max_width = 350).add_to(geo_j)
  geo_j.add_to(fg)

#add layer controls
folium.LayerControl().add_to(map_vernon)
map_vernon

In [ ]:
#work out the icons?
#add jobs, demographics, housing?
#FAILED NLCD LANDUSE EXPERIMENT

# vernon_outline.index = ["Vernon"]
# lulc = gh.nlcd_bygeom(vernon_outline, 100, years={"cover": [2021]})
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))

# cmap, norm, levels = gh.plot.cover_legends()
# cover = lulc["Vernon"].cover_2021
# cover
# cover.where(cover < 127).plot(ax=ax1, cmap=cmap, levels=levels, cbar_kwargs={"ticks": levels[:-1]})
# ax1.set_title("Land Use/Land Cover 2019")
# ax1.set_axis_off()
# cover_df = cover.to_dataframe().reset_index()
# cover_df["color"] = cover_df["cover_2021"].apply(cmap)
# geo_cover_df = gpd.GeoDataFrame(
#     cover_df, geometry=gpd.points_from_xy(cover_df.y, cover_df.x), crs="EPSG:4326"
# )
# geo_cover_df
# geo_cover_df = geo_cover_df.to_crs(espg=3395)
# buffer = geo_cover_df.buffer(30, cap_style=3)
# for _, r in cover_df.iterrows():
#     sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
#     geo_j = sim_geo.to_json()
#     geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "transparent"})
#     geo_j.add_to(map_vernon)

In [ ]:
map_vernon.save("VernonCO_chargerMap.html")